In [ ]:
!pip install pyyaml h5py

In [ ]:
# preparaing the dataset
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
image_size = 299
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
data_generator_train = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 25,
    zoom_range = 0.3,
    width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.2,
	horizontal_flip=True,
	fill_mode="nearest"
)

In [ ]:
# train data generator
train_generator = data_generator_train.flow_from_directory(
    '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/dataset/set1/train',
    target_size=(image_size, image_size),
    batch_size=20,
    class_mode='categorical',
    shuffle=True,
    seed=42
)
        

Found 5000 images belonging to 10 classes.


In [ ]:
# validation data
validation_generator = data_generator.flow_from_directory(
    '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/dataset/set1/validation',
    target_size=(image_size, image_size),
    batch_size=1,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Found 1000 images belonging to 10 classes.


In [ ]:
# test data
test_generator = data_generator.flow_from_directory(
    "/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/test",
    target_size=(image_size, image_size),
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 10 images belonging to 1 classes.


In [ ]:
test2_generator = data_generator.flow_from_directory(
    '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/validation sets/validation3',
    target_size=(image_size, image_size),
    batch_size=1,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Found 200 images belonging to 10 classes.


In [ ]:
# creating model for training
from keras.applications import InceptionV3
inceptionV3_weights = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
inceptionV3_model = InceptionV3(include_top=False, pooling='avg', weights=inceptionV3_weights, input_shape=(299, 299, 3))
x = inceptionV3_model.output
inceptionV3_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.optimizers import RMSprop, SGD, Adam, Adagrad
num_classes = 10
new_model = Sequential()
new_model.add(inceptionV3_model)

new_model.add(Flatten())
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(512, activation="relu"))
new_model.add(Dense(num_classes, activation='softmax'))

new_model.layers[0].trainable = False
new_model.compile(optimizer="sgd", loss='categorical_crossentropy', metrics=['accuracy'])
new_model.summary()

# from keras.models import Sequential
# from keras.layers import Dense, Flatten, GlobalAveragePooling2D
# from keras.optimizers import RMSprop, SGD, Adam, Adagrad
# num_classes = 10
# x = inceptionV3_model.output
# x = 
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation="relu")(x)
# predictions = Dense(2, activation="softmax")(x)
# new_model = Model(inputs=inceptionV3_model.input, outputs=predictions)
# new_model.layers[0].trainable = False
# new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
# new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 24,430,890
Trainable params: 2,628,106
Non-trainable params: 21,802,784
____________________________________

In [ ]:
# callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint
callback = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="max",
    baseline=0.97,
    restore_best_weights=False,
)

best_model_file = "/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/Fine tuning/InceptionV3/iv3_best_model.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_accuracy', mode='max',verbose=1, save_best_only=True)


In [ ]:
from keras.models import load_model
mymodel = load_model('/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/Fine tuning/InceptionV3/iv3_best_model.h5')

In [ ]:
# training the model
Step_size_train = train_generator.n // train_generator.batch_size
Step_size_valid = validation_generator.n // validation_generator.batch_size

history = mymodel.fit(
    train_generator,
    steps_per_epoch=Step_size_train,
    validation_data=validation_generator,
    validation_steps=Step_size_valid,
    epochs=15,
    verbose=1,
    callbacks=[best_model]
)

Epoch 1/15
250/250 [==============================] - 1753s 7s/step - loss: 0.1410 - accuracy: 0.9602 - val_loss: 0.1284 - val_accuracy: 0.9660

Epoch 00001: val_accuracy improved from -inf to 0.96600, saving model to /content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/Fine tuning/InceptionV3/iv3_best_model.h5
Epoch 2/15
250/250 [==============================] - 129s 514ms/step - loss: 0.1183 - accuracy: 0.9654 - val_loss: 0.1236 - val_accuracy: 0.9680

Epoch 00002: val_accuracy improved from 0.96600 to 0.96800, saving model to /content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/Fine tuning/InceptionV3/iv3_best_model.h5
Epoch 3/15
250/250 [==============================] - 128s 514ms/step - loss: 0.1213 - accuracy: 0.9610 - val_loss: 0.1218 - val_accuracy: 0.9650

Epoch 00003: val_accuracy did not improve from 0.96800
Epoch 4/15
250/250 [==============================] - 127s 508ms/step - loss: 0.1132 - accuracy: 0.9656 - val_loss: 0.1302 - val_accuracy: 0.9610



In [ ]:
# training the model
Step_size_train = train_generator.n // train_generator.batch_size
Step_size_valid = validation_generator.n // validation_generator.batch_size

history = new_model.fit(
    train_generator,
    steps_per_epoch=Step_size_train,
    validation_data=validation_generator,
    validation_steps=Step_size_valid,
    epochs=30,
    verbose=1,
    callbacks=[best_model]
)

In [ ]:
 # plotting the accuracy and loss curves woth epochs
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')

plt.legend()

plt.show()

In [ ]:
# evaluate the model
Step_size_test2 = test2_generator.n // test2_generator.batch_size
result = new_model.evaluate(test2_generator, steps=Step_size_test2)
print(result)
print(new_model.metrics_names)

200/200 [==============================] - 3s 16ms/step - loss: 0.3142 - accuracy: 0.9100
[0.3141677677631378, 0.9100000262260437]
['loss', 'accuracy']


In [ ]:
# saving the model
new_model.save('/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/models/Img1000/ResNet50/resnet50_set2_val96_test89.h5')

In [ ]:
# making predictions with the model
classes = train_generator.class_indices
print(classes)
labels = {}
for k, v in classes.items():
    labels[v] = k

print(labels)

{'butterfly': 0, 'cat': 1, 'chicken': 2, 'cow': 3, 'dog': 4, 'elephant': 5, 'horse': 6, 'sheep': 7, 'spider': 8, 'squirrel': 9}
{0: 'butterfly', 1: 'cat', 2: 'chicken', 3: 'cow', 4: 'dog', 5: 'elephant', 6: 'horse', 7: 'sheep', 8: 'spider', 9: 'squirrel'}


In [ ]:
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from PIL import Image 
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from keras.models import load_model

dog = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/dog.jpeg'
cat = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/cat.jpeg'
butterfly = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/butterfly.jpeg'
elephant = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/elephant.jpeg'
squirrel = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/squirrel.jpeg' 
spider = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/spider.jpeg'
cow = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/cow.jpeg'
chicken = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/chicken.jpeg'
horse = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/horse.jpeg'
sheep = '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/basic_images/sheep.jpeg'

imgs = [dog, cat, butterfly, elephant, squirrel, spider, cow, chicken, horse, sheep]

def _load_image(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

for img_path in imgs:
    img = _load_image(img_path)
    
    imag = cv2.imread(img_path)
    imag = cv2.resize(imag, (150, 150))
    cv2_imshow(imag)

    pred = sample_model.predict(img)
    pred = pred.flatten()
    for i in range(0, 3):
        prediction = np.argmax(pred)
        print(f'Predicted label {labels[prediction]} {pred[prediction] * 100}')
        pred[prediction] = -1

    print('\n')
    # print(pred)
    

In [ ]:
from keras.applications.resnet50 import preprocess_input
data_generator1 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = data_generator1.flow_from_directory(
    '/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/test_another',
    target_size=(image_size, image_size),
    batch_size=1,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Found 200 images belonging to 10 classes.


In [ ]:
from keras.models import load_model
mymodel = load_model('/content/drive/MyDrive/Google Colab Notebooks/DMW/Project Code/Fine tuning/InceptionV3/iv3_best_model.h5')

In [ ]:
# evaluate the model
print("Evaluating the model")
Step_size_test = test_generator.n // test_generator.batch_size
result = sample_model.evaluate(test_generator, steps=Step_size_test)
print(result)
print(sample_model.metrics_names)
print(f"Accuracy for validation set2 {result[1]}")
print(f"Loss for validation set2 {result[0]}")

Evaluating the model
200/200 [==============================] - 3s 16ms/step - loss: 0.1406 - accuracy: 0.9500
[0.1405780166387558, 0.949999988079071]
['loss', 'accuracy']
Accuracy for validation set2 0.949999988079071
Loss for validation set2 0.1405780166387558


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
num_classes = 10
new_model = Sequential()
new_model.add(resnet50_model)
new_model.add(GlobalAveragePooling2D())
new_model.add(Dropout(0.3))
new_model.add(Dense(num_classes, activation='softmax'))


new_model.layers[0].trainable = False
new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])